In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
raw_text = open('F:/JulyNLP/DLinNLP/DLinNLP/input/Winston_Churchil.txt',encoding='utf-8').read()
raw_text = raw_text.lower()  #转化为小写字母
raw_text

'\ufeffproject gutenberg’s real soldiers of fortune, by richard harding davis\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  you may copy it, give it away or\nre-use it under the terms of the project gutenberg license included\nwith this ebook or online at www.gutenberg.org\n\n\ntitle: real soldiers of fortune\n\nauthor: richard harding davis\n\nposting date: february 22, 2009 [ebook #3029]\nlast updated: september 26, 2016\n\nlanguage: english\n\ncharacter set encoding: utf-8\n\n*** start of this project gutenberg ebook real soldiers of fortune ***\n\n\n\n\nproduced by david reed, and ronald j. wilson\n\n\n\n\n\nreal soldiers of fortune\n\n\nby richard harding davis\n\n\n\n\n\nmajor-general henry ronald douglas maciver\n\nany sunny afternoon, on fifth avenue, or at night in the _table d’hote_\nrestaurants of university place, you may meet the soldier of fortune who\nof all his brothers in arms now living is the most remarkable.

In [3]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))  #value and index
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [4]:
chars

['\n',
 ' ',
 '!',
 '#',
 '$',
 '%',
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '‘',
 '’',
 '“',
 '”',
 '\ufeff']

In [5]:
len(chars)

61

In [6]:
len(raw_text)

276830

In [7]:
seq_length = 100
x = []
y = []  #前100个数是x,第101个数是y,这样循环
for i in range(0, len(raw_text) - seq_length):
    given = raw_text[i:i + seq_length]  #把100个取出来
    predict = raw_text[i + seq_length]  #第101个当作predict
    x.append([char_to_int[char] for char in given])
    y.append(char_to_int[predict])

In [8]:
print(x[:3])
print(y[:3])

[[60, 45, 47, 44, 39, 34, 32, 49, 1, 36, 50, 49, 34, 43, 31, 34, 47, 36, 57, 48, 1, 47, 34, 30, 41, 1, 48, 44, 41, 33, 38, 34, 47, 48, 1, 44, 35, 1, 35, 44, 47, 49, 50, 43, 34, 9, 1, 31, 54, 1, 47, 38, 32, 37, 30, 47, 33, 1, 37, 30, 47, 33, 38, 43, 36, 1, 33, 30, 51, 38, 48, 0, 0, 49, 37, 38, 48, 1, 34, 31, 44, 44, 40, 1, 38, 48, 1, 35, 44, 47, 1, 49, 37, 34, 1, 50, 48, 34, 1, 44], [45, 47, 44, 39, 34, 32, 49, 1, 36, 50, 49, 34, 43, 31, 34, 47, 36, 57, 48, 1, 47, 34, 30, 41, 1, 48, 44, 41, 33, 38, 34, 47, 48, 1, 44, 35, 1, 35, 44, 47, 49, 50, 43, 34, 9, 1, 31, 54, 1, 47, 38, 32, 37, 30, 47, 33, 1, 37, 30, 47, 33, 38, 43, 36, 1, 33, 30, 51, 38, 48, 0, 0, 49, 37, 38, 48, 1, 34, 31, 44, 44, 40, 1, 38, 48, 1, 35, 44, 47, 1, 49, 37, 34, 1, 50, 48, 34, 1, 44, 35], [47, 44, 39, 34, 32, 49, 1, 36, 50, 49, 34, 43, 31, 34, 47, 36, 57, 48, 1, 47, 34, 30, 41, 1, 48, 44, 41, 33, 38, 34, 47, 48, 1, 44, 35, 1, 35, 44, 47, 49, 50, 43, 34, 9, 1, 31, 54, 1, 47, 38, 32, 37, 30, 47, 33, 1, 37, 30, 47, 33,

In [9]:
n_patterns = len(x)
n_vocab = len(chars)

# 把x变成LSTM需要的样子 [样本数（len(x)，时间步伐(seq_length)，特征(1维特征)]
x = numpy.reshape(x, (n_patterns, seq_length, 1))
# 简单normal到0-1之间
x = x / float(n_vocab)
# output变成one-hot
y = np_utils.to_categorical(y)

print(x[11])
print(y[11])

[[ 0.80327869]
 [ 0.55737705]
 [ 0.70491803]
 [ 0.50819672]
 [ 0.55737705]
 [ 0.7704918 ]
 [ 0.59016393]
 [ 0.93442623]
 [ 0.78688525]
 [ 0.01639344]
 [ 0.7704918 ]
 [ 0.55737705]
 [ 0.49180328]
 [ 0.67213115]
 [ 0.01639344]
 [ 0.78688525]
 [ 0.72131148]
 [ 0.67213115]
 [ 0.54098361]
 [ 0.62295082]
 [ 0.55737705]
 [ 0.7704918 ]
 [ 0.78688525]
 [ 0.01639344]
 [ 0.72131148]
 [ 0.57377049]
 [ 0.01639344]
 [ 0.57377049]
 [ 0.72131148]
 [ 0.7704918 ]
 [ 0.80327869]
 [ 0.81967213]
 [ 0.70491803]
 [ 0.55737705]
 [ 0.14754098]
 [ 0.01639344]
 [ 0.50819672]
 [ 0.8852459 ]
 [ 0.01639344]
 [ 0.7704918 ]
 [ 0.62295082]
 [ 0.52459016]
 [ 0.60655738]
 [ 0.49180328]
 [ 0.7704918 ]
 [ 0.54098361]
 [ 0.01639344]
 [ 0.60655738]
 [ 0.49180328]
 [ 0.7704918 ]
 [ 0.54098361]
 [ 0.62295082]
 [ 0.70491803]
 [ 0.59016393]
 [ 0.01639344]
 [ 0.54098361]
 [ 0.49180328]
 [ 0.83606557]
 [ 0.62295082]
 [ 0.78688525]
 [ 0.        ]
 [ 0.        ]
 [ 0.80327869]
 [ 0.60655738]
 [ 0.62295082]
 [ 0.78688525]
 [ 0.01639

In [10]:
model = Sequential()
model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2]))) #神经元个数是256个,input shape考虑时间长度和特征量
model.add(Dropout(0.2)) #防止过拟合
model.add(Dense(y.shape[1], activation='softmax'))      #dense是指的一般的神网
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
model.fit(x, y, nb_epoch=5, batch_size=100)  #每100个数据跑一圈

Epoch 1/5
   100/276730 [..............................] - ETA: 543s - loss: 4.0828

D:\anaconda3\py3\lib\site-packages\keras\models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


276730/276730 [==============================] - 520s - loss: 2.9009   
Epoch 2/5
276730/276730 [==============================] - 520s - loss: 2.7614   
Epoch 3/5
276730/276730 [==============================] - 520s - loss: 2.7058   
Epoch 4/5
276730/276730 [==============================] - 520s - loss: 2.6581   
Epoch 5/5
276730/276730 [==============================] - 519s - loss: 2.6104   


In [15]:
#预测输出
def predict_next(input_array):
    x = numpy.reshape(input_array, (1, seq_length, 1))
    x = x / float(n_vocab)
    y = model.predict(x)
    return y

#自然语言转化为char to int
def string_to_index(raw_input):
    res = []
    for c in raw_input[(len(raw_input)-seq_length):]:
        res.append(char_to_int[c])
    return res
#把y 转化为one hot
def y_to_char(y):
    largest_index = y.argmax()
    c = int_to_char[largest_index]
    return c

In [16]:
def generate_article(init, rounds=200):
    in_string = init.lower()
    for i in range(rounds):
        n = y_to_char(predict_next(string_to_index(in_string)))
        in_string += n
    return in_string

In [17]:
init = 'His object in coming to New York was to engage officers for that service. He came at an opportune moment'
article = generate_article(init)
print(article)

his object in coming to new york was to engage officers for that service. he came at an opportune moment oo the coast of the care of the care of the care of the care of the care of the care of the care of the care of the care of the care of the care of the care of the care of the care of the care of the
